# Titanic Project!

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
#sns.set(font_scale=2.5)
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

train_df = pd.read_csv('/kaggle/input/titanic/train.csv')
test_df = pd.read_csv('/kaggle/input/titanic/test.csv')
submission_df = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

In [ ]:
print(train_df.shape)
print(test_df.shape)

#  **Dataset Check** 

In [ ]:
train_df.head()
# PassengerID: 탑승자 데이터 번호
# survived : 생존 1, 사망 0
# Pclass: 선실등급 1등석~3등석
# Sex: 성별
# Name: 이름
# Age: 나이
# SibSp: 같이 탑승한 형제자매, 배우자 인원
# parch: 같이 탑승한 부모, 어린이 인원수
# ticket: 티켓 번호
# Fare: 요금
# Cabin: 선실번호
# Embarked: 중간정착항구 약어 C, Q, S

In [ ]:
train_df.describe()

In [ ]:
train_df.info()

In [ ]:
train_df.isnull().sum()

* Age, Cabin, Embarked 는 Null Data를 가짐 
* sklearn은 Null Data 시 오류가 나므로 결측치를 채워줘야 함 

# **Feature Engineering**

# **1. How many Survived**

In [ ]:
sns.barplot(x='Sex', y='Survived', data=train_df);

여성이 남성보다 4배정도 생존률이 높은 것으로 확인됨.  
이는 여성과 어린이를 먼저 탈출시키는 방식이 반영된 결과로 해석

In [ ]:
sns.barplot(x='Pclass', y='Survived', hue='Sex', data=train_df);

더 높은 등석으로 갈수록 생존률이 높아짐 

In [ ]:
def get_name(age):
    if (age<=5): return 'Baby'
    elif (age<=13): return 'Child'
    elif (age<=19): return 'Teen'
    elif (age<=25): return 'Student'
    elif (age<=35): return 'Young'
    elif (age<=55): return 'Adult'
    else: return 'Elder'


plt.figure(figsize=(10,6))
group_names=["Baby","Child","Teen", "Student","Young", "Adult", "Elder"] # group name order
train_df['Age_cat']=train_df['Age'].apply(lambda x: get_name(x))
sns.barplot(x='Age_cat', y='Survived', hue='Sex', data=train_df, order=group_names)
train_df.drop('Age_cat', axis=1, inplace=True)

어린아이와 여성의 생존률이 높음을 확인할 수 있음

# **2. Analysing The Features**

## **Feature 1: Sex**

In [ ]:
# sex & survived
train_df.groupby(['Sex', 'Survived'])['Survived'].count()


In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
train_df[['Sex','Survived']].groupby(['Sex']).mean().plot.bar(ax=ax[0])
ax[0].set_title('fig1')
sns.countplot('Sex',hue='Survived',data=train_df,ax=ax[1])
ax[1].set_title('fig2')
plt.show()

*  성(Sex)이 생존(Survived)에 중요한 영향을 미치는 변수임을 알 수 있음(fig1)  
*  남성과 여성의 생사여부가 정반대로 나타나는 결과(fig2)

## **Feature 2: Pclass**

In [ ]:
pd.crosstab(train_df.Pclass,train_df.Survived,margins=True).style.background_gradient() # margins: 총합

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
train_df['Pclass'].value_counts().plot.bar(ax=ax[0])
ax[0].set_title('Number Of Passengers By Pclass')
ax[0].set_ylabel('Count')
sns.countplot('Pclass',hue='Survived',data=train_df,ax=ax[1])
ax[1].set_title('Pclass:Survived vs Dead')
plt.show()

In [ ]:
sns.factorplot('Pclass','Survived',hue='Sex',data=train_df)
plt.show()

* factorplot 함수는 범주형 데이터 시각화에 유리함  
* Pclass 역시 생존에 중요한 영향을 끼치는 feature임을 확인

## **Feature 3: Age**

In [ ]:
print('Oldest Passenger was of:',train_df['Age'].max(),'Years')
print('Youngest Passenger was of:',train_df['Age'].min(),'Years')
print('Average Age on the ship:',round(train_df['Age'].mean(),2),'Years')

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
sns.violinplot("Pclass","Age", hue="Survived", data=train_df,split=True,ax=ax[0])
ax[0].set_title('Pclass and Age vs Survived')
ax[0].set_yticks(range(0,110,10))
sns.violinplot("Sex","Age", hue="Survived", data=train_df,split=True,ax=ax[1])
ax[1].set_title('Sex and Age vs Survived')
ax[1].set_yticks(range(0,110,10))
plt.show()

*  Pclass가 높을수록 생존비율이 높아짐    
*  여자는 생존비율이 높고 남자는 사망비율이 높음  
*  남녀 모두 20~30대가 생존과 사망률이 비슷하게 분포

### **Age 결측치 데이터 채우기**
문제점: Age에는 177개의 Null Data가 있었음  
해결책: 평균나이로 채우기

In [ ]:
train_df['Age'].fillna(train_df['Age'].mean(), inplace=True)

## **Feature 4: Embarked**

In [ ]:
f,ax=plt.subplots(2,2,figsize=(20,10))

train_df['Embarked'].value_counts().plot.bar(ax=ax[0,0])
ax[0,0].set_title('fig 1: count of Embarked')

sns.countplot('Embarked',hue='Survived',data=train_df,ax=ax[0,1])
ax[0,1].set_title('fig 2: Embarked vs Survived')

sns.countplot('Embarked',hue='Sex',data=train_df,ax=ax[1,0])
ax[1,0].set_title('fig 3: Embarked vs Sex')

sns.countplot('Embarked',hue='Pclass',data=train_df,ax=ax[1,1])
ax[1,1].set_title('fig 4: Embarked vs Pclass')

plt.subplots_adjust(wspace=0.2,hspace=0.5)

plt.show()

* S, C, Q 중 S가 매우 높은 비중을 차지
* Sex와 Survived 모두 S,C,Q에서 유사한 비율을 차지함

### **Embarked 결측치 채우기**
2개밖에 없으므로 3/4 이상 차지하는 S로 채워줌

In [ ]:
train_df['Embarked'].fillna('S', inplace=True)
train_df.Embarked.isnull().any()

## **Feature 5: SibSp**

In [ ]:
pd.crosstab([train_df.SibSp],train_df.Survived, margins=True).style.background_gradient()

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
train_df['SibSp'].value_counts().plot.bar(ax=ax[0])
ax[0].set_title('fig 1')
ax[0].set_ylabel('Count')
sns.barplot('SibSp','Survived',data=train_df,ax=ax[1])
ax[1].set_title('fig 2')
plt.show()

*  SibSp가 0명인 승객이 67%이고 1명인 승객이 23%로 둘이 약 90%를 차지함
*  형제, 자매가 1~2명 있는 경우 0명인 사람보다 높은 생존률을 보임

## **Feature 6: Parch**

In [ ]:
pd.crosstab([train_df.Parch],train_df.Survived, margins=True).style.background_gradient()

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
train_df['Parch'].value_counts().plot.bar(ax=ax[0])
ax[0].set_title('fig 1')
ax[0].set_ylabel('Count')
sns.barplot('Parch','Survived',data=train_df,ax=ax[1])
ax[1].set_title('fig 2')
plt.show()

* Parch 역시 0과 1 값이 주를 이루고 있음
* Parch 역시 SibSp과 마찬가지로 1명이상인 경우가 0명보다 생존률이 높음 

## **Feature 7: Fare (Continous Feature)**

In [ ]:
print("Min Fare :", train_df['Fare'].min())
print("Max Fare :", train_df['Fare'].max())
print("Average Fare :", round(train_df['Fare'].mean(),2))

In [ ]:
f,ax=plt.subplots(1,3,figsize=(20,8))
sns.distplot(train_df[train_df['Pclass']==1].Fare,ax=ax[0])
ax[0].set_title('Fares in Pclass 1')
sns.distplot(train_df[train_df['Pclass']==2].Fare,ax=ax[1])
ax[1].set_title('Fares in Pclass 2')
sns.distplot(train_df[train_df['Pclass']==3].Fare,ax=ax[2])
ax[2].set_title('Fares in Pclass 3')
plt.show()

# **Feature Result** 

* **Sex**: 여성이 남성보다 생존률이 높음
* **Pclass**: 좋은 등석일수록 생존률이 높음
* **Age**: 어린아이(age<=5)가 생존률이 높고, 14~25세가 가장 많이 사망함
* **Embarked**: 'S'가 가장 많고 그 안에서 Pclass가 1<2<3등석 인원순으로 분포함 
* **Fare**: Pclass의 1,2,3등석에 따라 요금도 그에 상응하는 결과를 보임
* **SibSp,Parch**: 둘 모두 0명보다 1~3명정도 존재할 때 생존률이 높음

In [ ]:
sns.heatmap(train_df.corr(), annot=True, cmap='RdYlGn', linewidths=0.2)
fig=plt.gcf()
fig.set_size_inches(20,10)
plt.show()

*  SibSp와 Parch는 상관계수가 높음을 알 수 있음
*  상관계수가 매우 높은 두 feature면 둘 중 하나만 선택하는 것도 방법

# **데이터 전처리**

In [ ]:
# 불필요한 특성 제거
def drop(df):
    df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
    return df

# 문자형을 숫자로 변환 
def encode(df):
    df['Sex'].replace(['male','female'],[0,1],inplace=True)
    df['Embarked'].replace(['S','C','Q'],[0,1,2],inplace=True)
    return df

# 결측치 채우기
def fillna(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Embarked'].fillna('S', inplace=True)
    df['Fare'].fillna(df['Fare'].mean(), inplace=True)
    return df



In [ ]:
y_titanic=train_df['Survived']
X_titanic=train_df.drop('Survived', axis=1)
drop(X_titanic)
encode(X_titanic)
fillna(X_titanic)
X_titanic.head()


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X_titanic, y_titanic, test_size=0.2)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# **이진분류 수행(최적화 X)**

## 1. LogisticRegression

In [ ]:
# 로지스틱 회귀
lr=LogisticRegression()
lr.fit(X_train, y_train)
print("Train_scores: ", lr.score(X_train, y_train))
print("Test_scores: ", lr.score(X_test, y_test))

## 2. Decision Tree

In [ ]:
# 결정트리
dt=DecisionTreeClassifier(random_state=10)
dt.fit(X_train, y_train)
print("Train_scores: ", dt.score(X_train, y_train))
print("Test_scores: ", dt.score(X_test, y_test))
print(dt.feature_importances_)

## 3. Random Forest

In [ ]:
rf=RandomForestClassifier(random_state=10)
rf.fit(X_train, y_train)
print("Train_scores: ", rf.score(X_train, y_train))
print("Test_scores: ", rf.score(X_test, y_test))
print(dt.feature_importances_)

# **GridSearch**

## 1. Logistic Regression

In [ ]:
params={
    "C":[2, 4, 6, 8, 10],
    "max_iter":[100, 200, 300, 400, 500]
}
lr=LogisticRegression()
lr_grid=GridSearchCV(lr, param_grid=params, cv=5)
lr_grid.fit(X_train, y_train)
print(lr_grid.best_score_)
print(lr_grid.best_estimator_)

## 2. Random Forest

In [ ]:
params={"max_depth": [2, 3, 4, 5],
              "max_features": [1, 3, 5, 7],
              "min_samples_split": [2, 3, 4, 5],
              "min_samples_leaf": [1, 3, 5],
              "bootstrap": [False],
              "n_estimators" :[100,200,300],
              "criterion": ["gini"]
       }
rf=RandomForestClassifier(random_state=10)
gd=GridSearchCV(rf,param_grid=params,verbose=True, n_jobs=-1)
gd.fit(X_train,y_train)
print(gd.best_score_)
print(gd.best_estimator_)
# tunning model
rfc=gd.best_estimator_

In [ ]:
gd.best_params_

# **Testing**

In [ ]:
test_df.head()

In [ ]:
ID=test_df['PassengerId']
drop(test_df)
encode(test_df)
fillna(test_df)
print(test_df.head())

In [ ]:
print("Train_scores: ", rfc.score(X_train, y_train))
print("Test_scores: ", rfc.score(X_test, y_test))

In [ ]:
result = pd.DataFrame({ 'PassengerId': ID, 'Survived': rfc.predict(test_df) })
result.to_csv("titanic_RandomForest.csv", index=False)
result.head(10)
result.shape